In [1]:
import statsmodels
import scipy
from scipy import stats
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('churn_analysis.csv')

In [3]:
data.head()

Unnamed: 0 state  account_length  area_code intl_plan vmail_plan  \
0           0    KS             128        415        no        yes   
1           1    OH             107        415        no        yes   
2           2    NJ             137        415        no         no   
3           3    OH              84        408       yes         no   
4           4    OK              75        415       yes         no   

   vmail_message  day_mins  day_calls  day_charge   ...    night_mins  \
0             25     265.1        110       45.07   ...         244.7   
1             26     161.6        123       27.47   ...         254.4   
2              0     243.4        114       41.38   ...         162.6   
3              0     299.4         71       50.90   ...         196.9   
4              0     166.7        113       28.34   ...         186.9   

   night_calls  night_charge  intl_mins  intl_calls  intl_charge  \
0           91         11.01       10.0           3         2.70   
1          103         11.45       13.7           3         3.70   
2          104          7.32       12.2           5         3.29   
3           89          8.86        6.6           7         1.78   
4          121          8.41       10.1           3         2.73   

   custserv_calls  treatment  mes_estim   churn  
0               1          1       0.65  False.  
1               1          0       0.55  False.  
2               0          0       0.72  False.  
3               2          1       0.28  False.  
4               3          2       0.45  False.  

[5 rows x 23 columns]

In [4]:
print data.state.value_counts()[:3]
states = data.state.value_counts().index.values
print len(data[(data['state'] == states[0]) & (data['churn'] == 'False.')])
print len(data[(data['state'] == states[0]) & (data['churn'] == 'True.')])

WV    106
MN     84
NY     83
Name: state, dtype: int64
96
10


In [5]:
print np.zeros(2*2).reshape(2,2)

[[ 0.  0.]
 [ 0.  0.]]


In [8]:
control = data[data.treatment == 1]
subtable = np.zeros(2*2).reshape(2,2)
chi2_states = pd.DataFrame(columns=['state_pair', 'pvalue'])
chi2_states.set_index('state_pair')
for i in range(len(states)):
    for j in range(i + 1, len(states)):
        subtable[0][0] = len(control[(control['state'] == states[i]) & (control['churn'] == 'True.')])
        subtable[0][1] = len(control[(control['state'] == states[j]) & (control['churn'] == 'True.')])
        subtable[1][0] = len(control[(control['state'] == states[i]) & (control['churn'] == 'False.')])
        subtable[1][1] = len(control[(control['state'] == states[j]) & (control['churn'] == 'False.')])
        #print subtable[np.where(subtable < 5)[0],np.where(subtable < 5)[1]]
        chi2, p, dof, expected = scipy.stats.chi2_contingency(subtable, correction=False)
        chi2_states.loc[states[i] + '-' + states[j]] = [states[i] + '-' + states[j], p]
chi2_states.head()

state_pair    pvalue
WV-MN      WV-MN  0.521090
WV-NY      WV-NY  0.783432
WV-AL      WV-AL  0.911264
WV-WI      WV-WI  0.414216
WV-OH      WV-OH  0.414216

In [16]:
print len(chi2_states)
print (np.mean(chi2_states.pvalue))
print 'Answer 1:', len(chi2_states[chi2_states.pvalue < 0.05])

1275
0.501827379874
Answer 1: 34


2.
Применение критерия xи-квадрат для этих данных не обосновано, потому что не выполняются условия, при которых этот критерий дает правильные результаты. 

Интерпретация числа достигаемых уровней значимости, меньших α=0.05, некорректна, поскольку не сделана поправка на множественную проверку гипотез.

In [10]:

subtable = np.zeros(2*2).reshape(2,2)
chi2_states_corrected = pd.DataFrame(columns=['state_pair', 'pvalue'])
chi2_states_corrected.set_index('state_pair')
for i in range(len(states)):
    for j in range(i + 1, len(states)):
        subtable[0][0] = len(control[(control['state'] == states[i]) & (control['churn'] == 'True.')])
        subtable[0][1] = len(control[(control['state'] == states[j]) & (control['churn'] == 'True.')])
        subtable[1][0] = len(control[(control['state'] == states[i]) & (control['churn'] == 'False.')])
        subtable[1][1] = len(control[(control['state'] == states[j]) & (control['churn'] == 'False.')])
        #print subtable[np.where(subtable < 5)[0],np.where(subtable < 5)[1]]
        chi2, p, dof, expected = scipy.stats.chi2_contingency(subtable, correction=True)
        chi2_states_corrected.loc[states[i] + '-' + states[j]] = [states[i] + '-' + states[j], p]
chi2_states_corrected.head()

state_pair    pvalue
WV-MN      WV-MN  0.820478
WV-NY      WV-NY  0.955351
WV-AL      WV-AL  0.814959
WV-WI      WV-WI  0.683091
WV-OH      WV-OH  0.683091

In [15]:
print len(chi2_states_corrected)
print (np.mean(chi2_states_corrected.pvalue))
print 'Answer 3:', len(chi2_states_corrected[chi2_states_corrected.pvalue < 0.05])

1275
0.664056638205
Answer 3: 0


3.
Количество достигаемых уровней значимости, меньших, чем 0.05, в точности равно нулю. То есть поправка увеличила достигаемые уровни значимости настолько, что больше ни одно из значений достигаемого уровня значимости не попадает в диапазон от 0 до 0.05.

Достигаемые уровни значимости на наших данных, полученные с помощью критерия xи-квадрат с поправкой Йетса, в среднем получаются больше, чем соответствующие значения без поправки.

In [18]:
subtable = np.zeros(2*2).reshape(2,2)
fisher_states = pd.DataFrame(columns=['state_pair', 'pvalue'])
fisher_states.set_index('state_pair')
for i in range(len(states)):
    for j in range(i + 1, len(states)):
        subtable[0][0] = len(control[(control['state'] == states[i]) & (control['churn'] == 'True.')])
        subtable[0][1] = len(control[(control['state'] == states[j]) & (control['churn'] == 'True.')])
        subtable[1][0] = len(control[(control['state'] == states[i]) & (control['churn'] == 'False.')])
        subtable[1][1] = len(control[(control['state'] == states[j]) & (control['churn'] == 'False.')])
        #print subtable[np.where(subtable < 5)[0],np.where(subtable < 5)[1]]
        oddsratio, p = stats.fisher_exact(subtable)
        fisher_states.loc[states[i] + '-' + states[j]] = [states[i] + '-' + states[j], p]
fisher_states.head()

state_pair    pvalue
WV-MN      WV-MN  0.689725
WV-NY      WV-NY  1.000000
WV-AL      WV-AL  1.000000
WV-WI      WV-WI  0.686168
WV-OH      WV-OH  0.686168

In [19]:
print len(fisher_states)
print (np.mean(fisher_states.pvalue))
print 'Answer 4:', len(fisher_states[fisher_states.pvalue < 0.05])


1275
0.648338306002
Answer 4: 10


In [20]:
fisher_states.describe()

pvalue
count  1275.000000
mean      0.648338
std       0.311891
min       0.026963
25%       0.387902
50%       0.673781
75%       1.000000
max       1.000000

4.
Точный критерий Фишера всегда лучше, чем критерий xи-квадрат, потому что не использует аппроксимацию дискретного распределения непрерывным. Однако при увеличении размера выборки его преимущества по сравнению с критерем xи-квадрат уменьшаются, в пределе достигая нуля.

Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем значительно большие, чем xи-квадрат без поправки

Точный критерий Фишера на наших данных дает значения достигаемого уровня значимости в среднем меньшие, чем xи-квадрат с поправкой Йетса

In [30]:
from scipy.stats import pearsonr, spearmanr

In [28]:
subdata = data[['day_calls', 'mes_estim']]
day_calls_mes_estim_corr = subdata.corr()
corr, p = pearsonr(data[['day_calls']], data[['mes_estim']])
print ('corr = %f, pvalue = %f' % (corr, p))

corr = -0.051794, pvalue = 0.002780


5.Корреляция Пирсона имеет отрицательный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо. 

In [31]:
subdata = data[['day_calls', 'mes_estim']]
corr, p = spearmanr(data[['day_calls']], data[['mes_estim']])
print ('corr = %f, pvalue = %f' % (corr, p))

corr = 0.043350, pvalue = 0.012317


6.Корреляция Спирмена имеет положительный знак, и отличие корреляции от нуля на уровне доверия 0.05 значимо.

7.Посчитанные корреляции и их значимости говорят лишь о том, что необходимо взглянуть на данные глазами и попытаться понять, что приводит к таким (противоречивым?) результатам.

In [51]:
n1 = len(states)
actual = np.empty(shape=[2, n1])
for k1, s in enumerate(np.unique(control.state.values)):
    x = control[control.state == s]
    for k2,chu in enumerate(np.unique(control.churn.values)):
        actual[k2, k1] = len(x[x.churn == chu])
print actual
chi2, p, dof, expected = stats.chi2_contingency(actual)
print('p-value=%g') % (p)

[[ 19.  25.  11.  17.  10.  17.  25.  18.  16.  18.  19.  12.  13.  16.
   11.  20.  18.  21.  15.  17.  19.  20.  17.  19.  17.  20.  22.  16.
   21.  21.  10.  20.  27.  18.  27.  22.  11.  18.  14.  18.  16.  20.
   11.  14.  18.  20.  18.  16.  22.  27.  20.]
 [  1.   5.   5.   2.   5.   5.   5.   1.   1.   2.   2.   1.   2.   2.
    1.   3.   7.   4.   0.   7.   5.   8.   5.   2.   4.   4.   4.   3.
    3.   3.   2.   4.   2.   6.   6.   2.   2.   3.   4.   4.   5.   3.
    1.   6.   4.   3.   5.   7.   2.   5.   2.]]
p-value=0.709759


8.Для вычисления коэффициента Крамера используется значение статистики xи-квадрат, на которую мы не можем положиться применительно к нашим данным. 